In [26]:
import math
import random

In [27]:
def normal_cdf(x, mu=0,sigma=1):
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
    """find approximate inverse using binary search"""
    # if not standard, compute standard and rescale
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)
    low_z, low_p = -10.0, 0 # normal_cdf(-10) is (very close to) 0
    hi_z, hi_p = 10.0, 1 # normal_cdf(10) is (very close to) 1
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2 # consider the midpoint
        mid_p = normal_cdf(mid_z) # and the cdf's value there
        if mid_p < p:
        # midpoint is still too low, search above it
            low_z, low_p = mid_z, mid_p
        elif mid_p > p:
        # midpoint is still too high, search below it
            hi_z, hi_p = mid_z, mid_p
        else:
            break
    return mid_z
    

In [28]:
def normal_approximation_to_binomial(n, p):
    """finds mu and sigma corresponding to a Binomial(n, p)"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

# the normal cdf _is_ the probability the variable is below a threshold
normal_probability_below = normal_cdf

# it's above the threshold if it's not below the threshold
def normal_probability_above(lo, mu=0, sigma=1):
    return 1 - normal_cdf(lo, mu, sigma)

# it's between if it's less than hi, but not less than lo
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# it's outside if it's not between
def normal_probability_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

def normal_upper_bound(probability, mu=0, sigma=1): 
    """returns the z for which P(Z <= z) = probability""" 
    return inverse_normal_cdf(probability, mu, sigma)
def normal_lower_bound(probability, mu=0, sigma=1): 
    """returns the z for which P(Z >= z) = probability""" 
    return inverse_normal_cdf(1 - probability, mu, sigma)
def normal_two_sided_bounds(probability, mu=0, sigma=1): 
    """returns the symmetric (about the mean) bounds that contain the specified probability""" 
    tail_probability = (1 - probability) / 2
    
    # upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma) 
    # lower bound should have tail_probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma) 
    return lower_bound, upper_bound

mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)
print ("normal_approximation_to_binomial")
print ("mu_0      =",mu_0)
print ("sigma_0   =",sigma_0)
print (" ")
s_1, s_2 = normal_two_sided_bounds(0.95, mu_0, sigma_0) # (469, 531)
print ("normal_two_sided_bounds")
print ("s_1       = ",s_1)
print ("s_2       = ",s_2)

normal_approximation_to_binomial
mu_0      = 500.0
sigma_0   = 15.811388300841896
 
normal_two_sided_bounds
s_1       =  469.01026640487555
s_2       =  530.9897335951244


In [29]:
#95% bounds based on assumption p is 0.5
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)
print ("lo        = ",lo)
print ("hi        = ",hi)
# actual mu and sigma based on p = 0.55
print ("")
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)
print ("mu_1      = ",mu_1)
print ("sigma_1   = ",sigma_1)
print ("")
# a type 2 error means we fail to reject the null hypothesis
# which will happen when X is still in our original interval
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
print ("type_2_probability   = ",type_2_probability)
power = 1 - type_2_probability # 0.887
print ("power                = ",power)

lo        =  469.01026640487555
hi        =  530.9897335951244

mu_1      =  550.0
sigma_1   =  15.732132722552274

type_2_probability   =  0.11345199870463285
power                =  0.8865480012953671


In [30]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)
# is 526 (< 531, since we need more probability in the upper tail)
print ("hi        = ",hi) 
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
print ("type_2_probability   = ",type_2_probability)
power = 1 - type_2_probability # 0.936
print ("power                = ",power)

hi        =  526.0073585242053
type_2_probability   =  0.06362051966928273
power                =  0.9363794803307173


In [31]:
def two_sided_p_value(x, mu=0, sigma=1):
    if x >= mu:
        # if x is greater than the mean, the tail is what's greater than x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # if x is less than the mean, the tail is what's less than x
        return 2 * normal_probability_below(x, mu, sigma)

values = two_sided_p_value(529.5, mu_0, sigma_0) # 0.062
print ("two_sided_p_value   = ",values)

two_sided_p_value   =  0.06207721579598835


In [40]:
extreme_value_count = 0
for _ in range(100000):
    num_heads = sum(1 if random.random() < 0.5 else 0 # count # of heads
                    for _ in range(1000)) # in 1000 flips
    if num_heads >= 530 or num_heads <= 470 : # and count how often
        extreme_value_count += 1 # the # is 'extreme'
print ("extreme_value_count" , extreme_value_count)    
print (extreme_value_count / 100000) # 0.062
 



extreme_value_count 6137
0.06137


In [41]:
value = two_sided_p_value(531.5, mu_0, sigma_0) # 0.0463
print("two_sided_p_value    =", value)
print("")
 
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below 
print("upper_p_value    =", upper_p_value)
print("lower_p_value    =", lower_p_value)
print("")

value = upper_p_value(524.5, mu_0, sigma_0) # 0.061
print("upper_p_value    =", value)
print("")

value = upper_p_value(526.5, mu_0, sigma_0) # 0.047
print("upper_p_value    =", value)



two_sided_p_value    = 0.046345287837786575

upper_p_value    = <function normal_probability_above at 0x10b3ab2e0>
lower_p_value    = <function normal_cdf at 0x10b481ee0>

upper_p_value    = 0.06062885772582072

upper_p_value    = 0.04686839508859242
